# Sample Notebook
This is a sample Jupyter Notebook uploaded to GitHub.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate some data
x = np.linspace(0, 10, 100)
y = np.sin(x)

# Plot
plt.plot(x, y, label='sin(x)')
plt.legend()
plt.show()